In [1]:
import timeit as ti
from numba import jit, prange
from multiprocessing import Pool
import numpy as np

In this file I tested the performance between linear and recursive / treelike kronecker multiplication. 

Note that these functions are implemented in `q_lab_toolbox.unitary_circuits` with the turnover point found in this experiment hardcoded.

In [2]:
# defining the functions as done in q_lab_toolbox.unitary_circuits.py
# minus the documentation and type hints

@jit(forceobj=True)
def kron_gates_l(single_gates):
    result = single_gates[0]
    for gate in single_gates[1:]:
        result = np.kron(result, gate)

    return result


@jit(parallel=True)
def kron_neighbours_even(single_gates):

    l, dims, _ = single_gates.shape

    double_gates = np.zeros( (l//2, dims**2, dims**2) )

    for i in prange(0, l//2):
        double_gates[i,:,:] = np.kron(single_gates[i*2], single_gates[i*2 + 1])

    return double_gates


def kron_neighbours2(single_gates):
    """Attempt with multiprocessing, but turns out to be much slower
    than using numba.jit w/ parallel=True"""

    l, r = single_gates[0::2, :, :], single_gates[1::2, :, :]

    with Pool() as p:
        doubles = p.starmap(np.kron, zip(l, r))

    return doubles

@jit(forceobj=True)
def kron_gates_t(single_gates):
    """Recursively multiply the neighbouring gates.
    When the block size gets below the turnover point the linear
    kron_gates_l is used as it is more efficient in this usecase."""
    TURNOVER = 3

    l = len(single_gates)

    if l > TURNOVER:
        if l % 2 == 0:
            return kron_gates_t(kron_neighbours_even(single_gates))
        return np.kron(kron_gates_t(kron_neighbours_even(single_gates[:-1, :, :])), single_gates[-1])

    return kron_gates_l(np.array(single_gates))

Now follows the timing experiment.

Expectation:
* `kron_gates_t` gates is faster when multiplying many gates due to parallel computations
* `kron_gates_l` is faster for multiplying a few gates together due to less overhead

This leads us with the task of finding at which point `kron_gates_t` outperforms `kron_gates_l`.
This is the turnover point.

In [3]:
# first define some testing data

# a test with small matrices
test_small = lambda tp : np.random.rand(tp, 2, 2)


In [4]:
verbose = True

for i in range(2, 10):
    test = test_small(i)

    tl = ti.timeit(lambda: kron_gates_l(test.copy()), number=100)
    tt = ti.timeit(lambda: kron_gates_t(test.copy()), number=100)

    if verbose:
        print(f"""Testing kron_gates_l for multiplying {i} gates \r
        {tl} """)

        print(f"""Testing kron_gates_t for multiplying {i} gates \r
        {tt} """)

    if tl < tt:
        print(f"PREFER LINEAR (i={i})")

    if tt < tl:
        print(f"PREFER RECURSIVE (i={i})")

C:\Users\Admin\AppData\Local\Temp\ipykernel_19132\2520788973.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "kron_gates_l" failed type inference due to: No implementation of function Function(<function kron at 0x000002E06701FB00>) found for signature:
 
 >>> kron(array(float64, 2d, C), array(float64, 2d, A))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'kron_impl': File: numba\np\linalg.py: Line 2785.
    With argument(s): '(array(float64, 2d, C), array(float64, 2d, A))':
   Rejected as the implementation raised a specific error:
     TypingError: np.linalg.kron only supports 'C' or 'F' layout input arrays. Received an input of layout 'A'.
  raised from c:\Users\Admin\Desktop\BFP\quantum-channel-approximation\venv\Lib\site-packages\numba\np\linalg.py:2726

During: resolving callee type: Function(<function kron at 0x000002E06701FB00>)
During: typing of call at C:\Users\A

Testing kron_gates_l for multiplying 2 gates 
        0.54207480000332 
Testing kron_gates_t for multiplying 2 gates 
        0.21177090000128374 
PREFER RECURSIVE (i=2)
Testing kron_gates_l for multiplying 3 gates 
        0.005069399951025844 
Testing kron_gates_t for multiplying 3 gates 
        0.00526120001450181 
PREFER LINEAR (i=3)
Testing kron_gates_l for multiplying 4 gates 
        0.008285400050226599 
Testing kron_gates_t for multiplying 4 gates 
        3.9377805999829434 
PREFER LINEAR (i=4)
Testing kron_gates_l for multiplying 5 gates 
        0.01168699999107048 
Testing kron_gates_t for multiplying 5 gates 
        0.01159509998979047 
PREFER RECURSIVE (i=5)
Testing kron_gates_l for multiplying 6 gates 
        0.01763990003382787 
Testing kron_gates_t for multiplying 6 gates 
        0.012430400005541742 
PREFER RECURSIVE (i=6)
Testing kron_gates_l for multiplying 7 gates 
        0.03284980001626536 
Testing kron_gates_t for multiplying 7 gates 
        0.01472420000

After trying this a few times it seems that settings the turnover point at 3 is fair

Lastly note that kron_neighbours2 (which uses multiprocessing) is slower than kron_neighbours_even (which uses numba)

In [5]:
verbose = True

# note only taking steps of 2 because the numba version
# only works for even input
for i in range(2, 2, 2):
    test = test_small(i)

    t2 = ti.timeit(lambda: kron_neighbours2(test.copy()), number=100)
    te = ti.timeit(lambda: kron_neighbours_even(test.copy()), number=100)

    if verbose:
        print(
            f"""Testing kron_neighbours2 for multiplying {i} gates \r
        {t2} """
        )

        print(
            f"""Testing kron_neighbours_even for multiplying {i} gates \r
        {te} """
        )

    if t2 < te:
        print(f"PREFER kron_neighbours2 (i={i})")
    elif te < t2:
        print(f"PREFER kron_neighbours_even (i={i})")
    else:
        print(f"EQUAL PERFORMANCE (i={i})")